# **Door Opening**

In the lecture, we've seen an example of a robot manipulating a cylindrical object in the DARPA Robotics Challenge.  Similarly, by using optimization-based inverse kinematics, we will solve a series of IK problems that can open a cupboard door. You can take a look at the [interactive_ik](https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/0762b167-402a-4362-9702-7d559f0e73bb/notebook/interactive_ik-b6b0708a94b340b7b17cedbcc3d2d053?secondary-sidebar-autoopen=true&secondary-sidebar=agent) notebook, which illustrates many of the concepts we go through in this notebook.

**Learning Objectives**
- Formulate inverse kinematics problems with a joint-centering cost, position constraints and orientation constraints.
- Apply IK to a real-world task of opening a door.

**What You'll Implement**
- Set up the IK problem for a door-opening task by specifying the cost function and constraints.
- Solve the IK program over a trajectory of poses to generate a sequence of joint configurations that accomplishes the door-opening motion.

## Setup and Imports

Let us first import our standard drake functionality

In [ ]:
import numpy as np
from pydrake.all import (
    ConstantVectorSource,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    Parser,
    PiecewisePolynomial,
    PiecewiseQuaternionSlerp,
    RigidTransform,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)
from pydrake.multibody import inverse_kinematics
from pydrake.trajectories import PiecewisePolynomial

from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import AddMultibodyTriad
from manipulation.station import LoadScenario, MakeHardwareStation, MakeMultibodyPlant
from manipulation.utils import FindResource

In [ ]:
# Start the visualizer.
meshcat = StartMeshcat()

## Part 1: Visualizing the end effector trajectory

The goal of this notebook is to command the iiwa to open a cupboard door. We choose to open the door on the right. In the cell below, we provided you with setpoints ${}^W X^G(t)$ in the end-effector space, which gives us a nominal trajectory to reach and open the cupboard. "Nominal" refers to an ideal or baseline quantity. If you open the meshcat link from the cell above, and then run the cell below, you will be able to visualize the trajectory. This trajectory was computed by dividing the motion into following segments:

- $0 \leq t \leq 5$: The end-effector linearly interpolates between the initial pose at $t=0$, and the grasp pose required to grip the cylinder (${}^W\mathbf{X}^H$ at $t=5$), while having the gripper open.
- $5 \leq t \leq 6$: The end-effector stays still at ${}^W\mathbf{X}^H$, and the gripper is closed.
- $6 \leq t \leq 11$: The end-effector follows an arc of the handle as the door opens. The gripper remains closed.

In this notebook, you will convert this end effector trajectory to a joint-space trajectory using optimization!

In [ ]:
def setup_manipulation_station() -> RigidTransform:
    builder = DiagramBuilder()
    scenario = LoadScenario(filename=FindResource("models/cupboard.scenario.yaml"))
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    iiwa_position = builder.AddSystem(ConstantVectorSource(np.zeros(7)))
    builder.Connect(
        iiwa_position.get_output_port(), station.GetInputPort("iiwa.position")
    )

    wsg_position = builder.AddSystem(ConstantVectorSource([0.06]))
    builder.Connect(
        wsg_position.get_output_port(), station.GetInputPort("wsg.position")
    )

    diagram = builder.Build()

    context = plant.CreateDefaultContext()
    gripper = plant.GetBodyByName("body")

    initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

    simulator = Simulator(diagram)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(0.01)

    return initial_pose


# Get initial pose of the gripper by using default context of manip station.
initial_pose = setup_manipulation_station()

p_WR = np.array([0.7477, -0.1445, 0.4148])  # frame R: center of left door.

p_Rhandle = np.array([-0.033, 0.1245, 0])  # handle: frame attached to right handle.
p_Whandle = p_WR + p_Rhandle

p_Rhinge = np.array([0.008, -0.1395, 0])  # hinge: frame attached to right hinge.
p_Whinge = p_WR + p_Rhinge

p_Rhinge_handle = p_Rhandle - p_Rhinge
r_Rhinge_handle = np.linalg.norm(
    p_Rhandle - p_Rhinge
)  # distance between handle and hinge.

theta_Rhinge_handle = np.arctan2(p_Rhinge_handle[1], p_Rhinge_handle[0])
angle_end = np.pi  # end of angle. Decrease to 120~160 deg for the easy version.


# Interpolate pose for opening doors.
def InterpolatePoseOpen(t: float) -> RigidTransform:
    # Start by interpolating the yaw angle of the hinge.
    angle_start = theta_Rhinge_handle
    theta = angle_start + (angle_end - angle_start) * t
    # Convert to position and rotation.
    p_Whandle = r_Rhinge_handle * np.array([np.cos(theta), np.sin(theta), 0]) + p_Whinge
    # Add some offset here to account for gripper yaw angle.
    R_Whandle = RollPitchYaw(0, 0, theta).ToRotationMatrix()
    X_Whandle = RigidTransform(R_Whandle, p_Whandle)

    # Add a little offset to account for gripper.
    p_handleG = np.array([0.0, 0.1, 0.0])
    R_handleG = RollPitchYaw(0, np.pi, np.pi).ToRotationMatrix()
    X_handleG = RigidTransform(R_handleG, p_handleG)
    X_WG = X_Whandle.multiply(X_handleG)
    return X_WG


## Interpolate Pose for entry.
def make_gripper_orientation_trajectory() -> PiecewiseQuaternionSlerp:
    traj = PiecewiseQuaternionSlerp()
    traj.Append(0.0, initial_pose.rotation())
    traj.Append(5.0, InterpolatePoseOpen(0.0).rotation())
    return traj


def make_gripper_position_trajectory() -> PiecewisePolynomial:
    traj = PiecewisePolynomial.FirstOrderHold(
        [0.0, 5.0],
        np.vstack(
            [
                [initial_pose.translation()],
                [InterpolatePoseOpen(0.0).translation()],
            ]
        ).T,
    )
    return traj


entry_traj_rotation = make_gripper_orientation_trajectory()
entry_traj_translation = make_gripper_position_trajectory()


def InterpolatePoseEntry(t: float) -> RigidTransform:
    return RigidTransform(
        RotationMatrix(entry_traj_rotation.orientation(t)),
        entry_traj_translation.value(t),
    )


# Wrapper function for end-effector pose. Total time: 11 seconds.
def InterpolatePose(t: float) -> RigidTransform:
    if t < 5.0:
        # Duration of entry motion is set to 5 seconds.
        return InterpolatePoseEntry(t)
    elif (t >= 5.0) and (t < 6.0):
        # Wait for a second to grip the handle.
        return InterpolatePoseEntry(5.0)
    else:
        # Duration of the open motion is set to 5 seconds.
        return InterpolatePoseOpen((t - 6.0) / 5.0)


# Visualize our end-effector nominal trajectory.
t_lst = np.linspace(0, 11, 30)
pose_lst = []
for t in t_lst:
    AddMeshcatTriad(meshcat, path=str(t), X_PT=InterpolatePose(t), opacity=0.2)
    pose_lst.append(InterpolatePose(t))

# Create gripper trajectory.
gripper_t_lst = np.array([0.0, 5.0, 6.0, 11.0])
gripper_knots = np.array([0.02, 0.02, 0.0, 0.0]).reshape(1, 4)
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_t_lst, gripper_knots)


def CreateIiwaControllerPlant() -> tuple[MultibodyPlant, list[int]]:
    """creates plant that includes only the robot and gripper, used for controllers."""
    scenario = LoadScenario(filename=FindResource("models/cupboard.scenario.yaml"))
    plant_robot = MakeMultibodyPlant(
        scenario=scenario, model_instance_names=["iiwa", "wsg"]
    )

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index()
        )

    return plant_robot, link_frame_indices


def BuildAndSimulateTrajectory(
    q_traj: Trajectory, g_traj: Trajectory, duration: float = 0.01
) -> tuple[Simulator, MultibodyPlant]:
    """Simulate trajectory for manipulation station.
    @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
    @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
    """
    builder = DiagramBuilder()
    scenario = LoadScenario(filename=FindResource("models/cupboard.scenario.yaml"))
    station = builder.AddSystem(MakeHardwareStation(scenario, meshcat))
    plant = station.GetSubsystemByName("plant")
    scene_graph = station.GetSubsystemByName("scene_graph")
    AddMultibodyTriad(plant.GetFrameByName("body"), scene_graph)

    q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
    g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

    builder.Connect(
        q_traj_system.get_output_port(), station.GetInputPort("iiwa.position")
    )
    builder.Connect(
        g_traj_system.get_output_port(), station.GetInputPort("wsg.position")
    )

    diagram = builder.Build()

    simulator = Simulator(diagram)
    meshcat.StartRecording(set_visualizations_while_recording=False)
    simulator.AdvanceTo(duration)
    meshcat.PublishRecording()

    return simulator, plant

## Part 2: Thought Exercise - Can we Relax Constraints?

For this thought exercise, we will be focusing on just our grasping pose keyframe, and disregarding the rest of the trajectory. We define $H$ as the frame the center of the handle, which we can observe in the diagram below. We will denote our decision variables as $q$ (joint angles of the arm), and the forward dynamics function as

$${}^W\mathbf{X}^G = f(q)$$

(You may access the rotation and translation parts independently with ${}^W\mathbf{R}^G=f_R(q)$ and ${}^Wp^G=f_p(q)$). 

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/book/figures/exercises/door.png" width="700">

Recall one of the main ideas from the lecture: instead of having ${}^W\mathbf{X}^G=f(q)$ be constrained to exactly ${}^W\mathbf{X}^H$, the inverse kinematics problem can benefit a lot by allowing a set of ${}^W\mathbf{X}^G$ which makes sense for our problem.





### Gradescope Verification 1
**Question:** Let us parametrize $${}^H X ^G$$ with six degrees of freedom: xyz positions and roll-pitch-yaw rotation (i.e. rotations around the xyz axii, respectively) of frame $G$ with respect to frame $H$. Which three should we constrain to be exactly equal, and which three should willing to provide some slack? For this, let's ignore the length of the grippers. In Gradescope, select the three which we should constraint to be **exactly equal**:

A. x position <br>
B. y position <br>
C. z position <br>
D. x orientation (roll) <br>
F. y orientation (pitch) <br>
G. z orientation (yaw) <br>

(HINT: You should constrain two positions and one orientation. Remember we are ignoring the length of the grippers. Also remember the x-y-z axes are represented in red-green-blue colors respectively.)

Now, for the purposes of our problem, we want to enforce equality constraints (up to a small tolerance) on both the position and the orientation. This is because we will be solving IK along an entire trajectory, not just for the grasp pose. That said, in other contexts we might choose to introduce slack or relaxations to improve feasibility.

We can write our final optimization along the entire trajectory. For each pose $I$, we solve:
$$
\begin{align*}
\min_q \quad & \lVert q - q_{nom} \rVert^2 \\
\text{s.t.} \quad & f_p(q) = {}^W p^I \\
& f_R(q) = {}^W R^I
\end{align*}
$$

### Gradescope Verification 2
**Question:** Recall that $q_{nom}$ are the baseline joint angles we define (and update) throughout our the optimization. What is the purpose of the quadratic cost term in our optimization formulation? Select all that apply

A. To bias the solver toward a preferred “natural” configuration when multiple solutions exist. <br>
B. To ensure joint limits are respected. <br>
C. To guarantee that orientation constraints are satisfied exactly. <br>
D. To improve numerical stability and avoid unnecessarily large joint displacements. <br>


## Part 3: Solving The Optimization

Now it's time to implement the optimization!

**YOUR TASK:** Below, you must implement `create_q_knots`, which accepts a list of key frames `pose_lst`, and converts them to joint coordinates. You will use Drake's `inverse_kinematics` package.

For each keyframe $I$, you must:
- Implement constraints on `ik` using `AddOrientationConstraint` and `AddPositionConstraint`. Note that these implement inequality constraints, which is okay if our error tolerance is small.
- Add a joint-centering cost on `q_nominal`.
- If `i==0`, set the initial guess to be nominal configuration using [`prog.SetInitialGuess`](https://drake.mit.edu/doxygen_cxx/classdrake_1_1solvers_1_1_mathematical_program.html#ae48cb6d2263ccf09e38932dcd27f769f). Otherwise, set the initial guess to be the solution you obtained on the previous IK problem.


Before you get started, go through these exercises to get an idea of how to implement the constraints in Drake's [InverseKinematics]() class.

### Gradescope Verification 3
Read the documentation for the [AddOrientationConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a693b3a9627e08ae92d2978b4ca516b3c) Method in Drake to answer the following questions. Note that AddOrientationConstraint implements an inequality constraint, which is okay for this problem if we set the theta bound to be small.

**Question:** What are frames $\bar{A}$ and $\bar{B}$? Select all that apply.

A. Arbitrary "virtual" frames that do not need to exist in the plant <br>
B. Frames existing in the MultibodyPlant <br>
C. Task frames constrained to have angle difference $\theta \leq \theta_{bound}$ <br>
D. Reference frames such that $^{\bar{A}}R^A$ and $^{\bar{B}}R^B$ are constant throughout the optimization <br>
E. Reference frames such that $^{\bar{A}}R^A$ and $^{\bar{B}}R^B$ may change throughout the optimization

**Question:** What is the purpose of frames $\bar{A}$ and $\bar{B}$? Select all that apply.

A. They are the frames which we wish to align <br>
B. Use existing frames in the MultibodyPlant to express the task frames $A$ and $B$ in <br>
C. Allow you to define orientation constraints on arbitrary axes that are rigidly attached to bodies <br>
D. Ensure that you only give the orientation of $A$ and $B$ relative to the world and gripper, respectively <br>

### Gradescope Verification 4
Read the documentation for the [AddPositionConstraint](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_inverse_kinematics.html#a7c669013ee2890d3a8c457668541cdba) Method in Drake to answer the following questions:

**Question:** Which of the following are true about the point $Q$? Select all that apply.

A. It is a point fixed relative to frame B<br>
B. It is an arbitrary free-floating point in space <br>
C. It changes position relative to frame B during optimization <br>
D. We aim to constrain its position in frame A <br>

**Question:** What does the constraint enforce?

A. That the coordinates of Q, expressed in frame B, remain between ${}^A p ^{Q\_lower}$ and ${}^A p ^{Q\_upper}$ <br>
B. That the distance between A and B is fixed <br>
D. That the coordinates of Q, expressed in frame A, remain between ${}^A p ^{Q\_lower}$ and ${}^A p ^{Q\_upper}$ <br>


Now you're ready! Complete the `create_q_knots` function below.

In [ ]:
def create_q_knots(pose_lst: list[RigidTransform]) -> np.ndarray:
    """Convert end-effector pose list to joint position list using series of
    InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions
    contain gripper joints, but these should not matter to the constraints.
    @param: pose_lst list[RigidTransform]: post_lst[i] contains keyframe X_WG at index i.
    @return: q_knots np.ndarray: q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
    """
    q_knots = []
    plant, _ = CreateIiwaControllerPlant()
    world_frame = plant.world_frame()
    gripper_frame = plant.GetFrameByName("body")
    q_nominal = np.array(
        [0.0, 0.6, 0.0, -1.75, 0.0, 1.0, 0.0, 0.0, 0.0]
    )  # nominal joint angles for joint-centering.

    for i in range(len(pose_lst)):
        ik = inverse_kinematics.InverseKinematics(plant)
        q_variables = ik.q()  # Get variables for MathematicalProgram
        prog = ik.prog()  # Get MathematicalProgram

        ### TODO: Add your constraints, cost, and initial guess here ###

        ################################################

        result = Solve(prog)

        assert result.is_success()

        q_knots.append(result.GetSolution(q_variables))

    return q_knots

When you have implemented your function, you can run the cell below to initialize your simulation. Once everything loads up in Meshcat, run the next cell to run the full simulation!

In [ ]:
q_knots = np.array(create_q_knots(pose_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, g_traj, 11.0)

You should now see the robot following the trajectory to open the door! You can also check the correctness of your implementation with the cell below.

In [ ]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.trajectories.test_door_opening import TestDoorOpening

Grader.grade_output([TestDoorOpening], [locals()], "results.json")
Grader.print_test_results("results.json")

### Gradescope Verification 5

Now that you have the list of optimal joint angles at each pose for opening the door, verify your implementation in Gradescope.

**Question:** In gradescope, enter the optimal joint angles for the final pose in the trajectory. Please give your answers to 4 decimal places. Recall that the last 2 dimensions of q are the gripper joints, so we can ignore those.